In [126]:
import pandas as pd
import sys
import praw
import json

In [2]:
reddit = praw.Reddit(
    client_id="pt5Jv1sjiMJcNQ",
    client_secret="szxNI44AB0JiI0BF5g6au4SFdsTV2A",
    user_agent="my user agent"
)

Version 7.1.2 of praw is outdated. Version 7.1.4 was released 7 days ago.


In [3]:
subreddit_female = reddit.subreddit("femalefashionadvice")
subreddit_male = reddit.subreddit("malefashionadvice")

In [235]:
def get_all_replies(comment):
    replies = []
    if comment.replies.__len__() != 0:
        for reply in comment.replies:
            reply_dict = {}
            reply_dict['author'] = reply.author.name
            reply_dict['id'] = reply.id
            reply_dict['body'] = reply.body
            reply_dict['score'] = reply.score
            reply_dict['ups'] = reply.ups
            reply_dict['downs'] = reply.downs
            reply_dict['date'] = reply.created
            reply_dict['depth'] = reply.depth
            reply_dict['replies'] = []
            replies.append(reply_dict)
            
            all_replies = get_all_replies(reply)
            if len(all_replies) > 0:
                replies[-1]['replies'] = all_replies
    return replies

In [236]:
SUBMISSION_LIMIT = 10 # None if we don't want limit
START_DATE = datetime.now() - timedelta(days=365)
END_DATE = datetime.now()
submissions = []
for submission in subreddit.new(limit=SUBMISSION_LIMIT): # you can also query them by 'hot' or 'top'
    json_output = {}
    if ((datetime.utcfromtimestamp(submission.created) < START_DATE) 
    | (datetime.utcfromtimestamp(submission.created) > END_DATE)):
        continue
    
    if ((submission.link_flair_text == '[Daily]') 
        | (submission.link_flair_text == '[Weekly]')):
        continue
    
    json_output['title'] = submission.title
    json_output['body'] = submission.selftext
    json_output['score'] = submission.score
    json_output['id'] = submission.id
    json_output['author'] = submission.author.name
    json_output['ups'] = submission.ups
    json_output['downs'] = submission.downs
    json_output['date'] = submission.created
    json_output['comments'] = []
    
    
    submission.comments.replace_more(limit=None)
    
    comments = submission.comments.list()
    
    for comment in submission.comments:
        comment_dict = {}
        comment_dict['author'] = comment.author.name
        comment_dict['id'] = comment.id
        comment_dict['body'] = comment.body
        comment_dict['score'] = comment.score
        comment_dict['ups'] = comment.ups
        comment_dict['downs'] = comment.downs
        comment_dict['date'] = comment.created
        comment_dict['depth'] = comment.depth
        
        comments = get_all_replies(comment)
        comment_dict['replies'] = comments
        json_output['comments'].append(comment_dict)
    submissions.append(json_output)
print(json.dumps(submissions, indent=3))

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readt

[
   {
      "title": "Alternative to bras?",
      "body": "Bras are a double edged sword. They give a nice shape, support and are very sexy. \n\nBut!! They\u2019re so uncomfortable sometimes and fit weird after a while. Fruit of the loom sports bras are great for scrubs, but what about when I wanna dress nicely? Can\u2019t be braless as a physician plus all that underboob sweat. \n\nAny recommendations out there? Maybe tops with built in support?",
      "score": 2,
      "id": "lk5gda",
      "author": "mintymindmatter",
      "ups": 2,
      "downs": 0,
      "date": 1613391399.0,
      "comments": [
         {
            "author": "laurasaurus5",
            "id": "gnk0hoz",
            "body": "I basically live in bralettes right now. In the summer I wear a lot of camisoles with built-in support. Traditional bras, in addition to being less comfortable, somehow also make me feel fatter and look fatter, and no amount of r/abrathatfits or department store specialists' assistance ha